# ⚽ Soccer Player Highlight Reel Generator - Production Grade

This notebook implements a complete, production-grade, end-to-end pipeline for generating personalized soccer player highlight reels. This is an advanced version with sophisticated algorithms for each stage.

## 🚀 Advanced Features
- Player Detection: High-performance YOLOv8 for person detection.
- Multi-Object Tracking: Full implementation of ByteTrack with a Kalman Filter for motion prediction and Hungarian Algorithm for association.
- Long-term Re-Identification: A hybrid system using a Deep CNN for appearance embeddings combined with color histograms and Jersey OCR.
- Intelligent Event Detection: Advanced kinematics-based event detector.
- Professional Video Assembly: Integration of PySceneDetect to find natural scene boundaries for clean clip extraction, stitched with FFmpeg.

## ⚡ How to Use
1. Enable GPU: Runtime → Change runtime type → T4 GPU
2. Run All Cells: Runtime → Run all
3. Upload Video when prompted
4. Wait for processing
5. Download results automatically


In [ ]:
print("Installing dependencies...")
!pip install ultralytics torch torchvision opencv-python-headless easyocr scikit-learn numpy pandas tqdm pillow 'scenedetect[opencv]' --quiet
import torch, os
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU detected - using CPU (will be much slower)")
os.makedirs('/content/videos', exist_ok=True)
os.makedirs('/content/output', exist_ok=True)
os.makedirs('/content/temp_clips', exist_ok=True)
print("Setup complete!")


In [ ]:
from google.colab import files
import shutil
print("Please upload your soccer match video file.")
uploaded = files.upload()
video_path = None
for filename in uploaded.keys():
    if filename.lower().endswith(('.mp4', '.avi', '.mov')):
        destination_path = f'/content/videos/{filename}'
        shutil.move(filename, destination_path)
        print(f"Video uploaded: {destination_path}")
        video_path = destination_path
        break
if not video_path:
    print("No video file found. Please upload an MP4, AVI, or MOV file.")


In [ ]:
import cv2
import torch
import numpy as np
import json
from ultralytics import YOLO
import easyocr
from tqdm.notebook import tqdm
import math
from sklearn.metrics.pairwise import cosine_similarity
import subprocess
from scipy.optimize import linear_sum_assignment
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Dict, Tuple
from scenedetect import open_video, SceneManager
from scenedetect.detectors import ContentDetector
print("Modules imported.")


In [ ]:
class SoccerPlayerDetector:
    def __init__(self, model_name: str = 'yolov8n.pt', conf_thresh: float = 0.35):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = YOLO(model_name)
        self.model.to(self.device)
        self.conf_thresh = conf_thresh
    def process_video(self, video_path: str, output_path: str) -> List[Dict]:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return []
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        all_detections = []
        with tqdm(total=total_frames, desc="Stage 1: Detecting players") as pbar:
            for frame_idx in range(total_frames):
                ret, frame = cap.read()
                if not ret:
                    break
                results = self.model(frame, classes=[0], imgsz=960, verbose=False)
                detections = []
                if len(results) > 0 and results[0].boxes is not None:
                    for box in results[0].boxes:
                        if box.conf[0] >= self.conf_thresh:
                            detections.append({'bbox': [int(coord) for coord in box.xyxy[0].tolist()], 'confidence': float(box.conf[0])})
                all_detections.append({"frame_id": frame_idx, "detections": detections})
                pbar.update(1)
        cap.release()
        with open(output_path, 'w') as f:
            json.dump(all_detections, f, indent=2)
        return all_detections


In [ ]:
from scipy.optimize import linear_sum_assignment

def _fixed_linear_assignment(self, cost_matrix, thresh):
    import numpy as np
    if getattr(cost_matrix, 'size', 0) == 0:
        rows = cost_matrix.shape[0] if hasattr(cost_matrix, 'shape') else 0
        cols = cost_matrix.shape[1] if hasattr(cost_matrix, 'shape') else 0
        return [], list(range(rows)), list(range(cols))
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    matches = [(r, c) for r, c in zip(row_ind, col_ind) if cost_matrix[r, c] < thresh]
    u_track = [r for r in range(cost_matrix.shape[0]) if r not in [m[0] for m in matches]]
    u_detection = [c for c in range(cost_matrix.shape[1]) if c not in [m[1] for m in matches]]
    return matches, u_track, u_detection

ByteTrack.linear_assignment = _fixed_linear_assignment


In [ ]:
class KalmanFilter:
    def __init__(self):
        self.kf = cv2.KalmanFilter(7, 4)
        self.kf.measurementMatrix = np.array([[1,0,0,0,0,0,0], [0,1,0,0,0,0,0], [0,0,1,0,0,0,0], [0,0,0,1,0,0,0]], np.float32)
        self.kf.transitionMatrix = np.array([[1,0,0,0,1,0,0], [0,1,0,0,0,1,0], [0,0,1,0,0,0,0], [0,0,0,1,0,0,1], [0,0,0,0,1,0,0], [0,0,0,0,0,1,0], [0,0,0,0,0,0,1]], np.float32)
        cv2.setIdentity(self.kf.processNoiseCov, 1e-2)
        cv2.setIdentity(self.kf.measurementNoiseCov, 1e-1)
        cv2.setIdentity(self.kf.errorCovPost, 1)
    def predict(self):
        return self.kf.predict()
    def update(self, bbox: List[float]):
        x, y, w, h = bbox
        measurement = np.array([x + w / 2, y + h / 2, w / h, h], dtype=np.float32).reshape(4, 1)
        self.kf.correct(measurement)
    def init(self, bbox: List[float]):
        x, y, w, h = bbox
        self.kf.statePost = np.array([x + w / 2, y + h / 2, w / h, h, 0, 0, 0], dtype=np.float32).reshape(7, 1)
class STrack:
    def __init__(self, tlwh: List[float], score: float):
        self.tlwh = np.asarray(tlwh, dtype=float)
        self.score = score
        self.kalman_filter = KalmanFilter()
        self.kalman_filter.init(self.tlwh)
        self.track_id = 0
        self.state = 'new'
        self.is_activated = False
        self.frame_id = 0
        self.start_frame = 0
        self.time_since_update = 0
    def activate(self, frame_id: int, track_id: int):
        self.track_id = track_id
        self.frame_id = frame_id
        self.start_frame = frame_id
        self.state = 'tracked'
        self.is_activated = True
    def re_activate(self, new_track, frame_id: int):
        self.tlwh = new_track.tlwh
        self.score = new_track.score
        self.kalman_filter.update(self.tlwh)
        self.state = 'tracked'
        self.is_activated = True
        self.frame_id = frame_id
        self.time_since_update = 0
    def predict(self):
        if self.state != 'tracked':
            self.kalman_filter.kf.statePost[6,0] = 0
        self.kalman_filter.predict()
    def update(self, new_track, frame_id: int):
        self.tlwh = new_track.tlwh
        self.score = new_track.score
        self.kalman_filter.update(self.tlwh)
        self.state = 'tracked'
        self.is_activated = True
        self.frame_id = frame_id
        self.time_since_update = 0
    @property
    def tlbr(self) -> List[float]:
        x, y, w, h = self.tlwh
        return [x, y, x + w, y + h]
def iou_distance(atracks: List[STrack], btracks: List[STrack]) -> np.ndarray:
    if not atracks or not btracks: return np.empty((0, 0))
    atlbrs = np.array([track.tlbr for track in atracks])
    btlbrs = np.array([track.tlbr for track in btracks])
    ious = np.zeros((len(atlbrs), len(btlbrs)))
    for i, a in enumerate(atlbrs):
        for j, b in enumerate(btlbrs):
            box_inter = [max(a[0], b[0]), max(a[1], b[1]), min(a[2], b[2]), min(a[3], b[3])]
            inter_area = max(0, box_inter[2] - box_inter[0]) * max(0, box_inter[3] - box_inter[1])
            union_area = (a[2] - a[0]) * (a[3] - a[1]) + (b[2] - b[0]) * (b[3] - b[1]) - inter_area
            if union_area > 0: ious[i, j] = inter_area / union_area
    return 1 - ious
class ByteTrack:
    def __init__(self, high_thresh: float = 0.6, low_thresh: float = 0.1, max_time_lost: int = 30):
        self.tracked_stracks: List[STrack] = []
        self.lost_stracks: List[STrack] = []
        self.removed_stracks: List[STrack] = []
        self.frame_id = 0
        self.track_id_count = 0
        self.high_thresh = high_thresh
        self.low_thresh = low_thresh
        self.max_time_lost = max_time_lost
    def update(self, detections: List[Dict]) -> List[Dict]:
        self.frame_id += 1
        activated_starcks, refind_stracks, lost_stracks, removed_stracks = [], [], [], []
        dets_high = [d for d in detections if d['confidence'] >= self.high_thresh]
        dets_low = [d for d in detections if self.low_thresh <= d['confidence'] < self.high_thresh]
        stracks_high = [STrack([*d['bbox'][:2], d['bbox'][2]-d['bbox'][0], d['bbox'][3]-d['bbox'][1]], d['confidence']) for d in dets_high]
        stracks_low = [STrack([*d['bbox'][:2], d['bbox'][2]-d['bbox'][0], d['bbox'][3]-d['bbox'][1]], d['confidence']) for d in dets_low]
        for strack in self.tracked_stracks: strack.predict()
        dists = iou_distance(self.tracked_stracks, stracks_high)
        matches, u_track, u_detection = self.linear_assignment(dists, 0.8)
        for i, j in matches:
            track = self.tracked_stracks[i]
            det = stracks_high[j]
            track.update(det, self.frame_id)
            activated_starcks.append(track)
        unmatched_tracks = [self.tracked_stracks[i] for i in u_track]
        dists = iou_distance(unmatched_tracks, stracks_low)
        matches, u_track, u_detection_low = self.linear_assignment(dists, 0.5)
        for i, j in matches:
            track = unmatched_tracks[i]
            det = stracks_low[j]
            track.update(det, self.frame_id)
            activated_starcks.append(track)
        for i in u_track:
            track = unmatched_tracks[i]
            track.state = 'lost'
            lost_stracks.append(track)
        for i in u_detection:
            track = stracks_high[i]
            if track.score >= self.high_thresh:
                self.track_id_count += 1
                track.activate(self.frame_id, self.track_id_count)
                activated_starcks.append(track)
        self.tracked_stracks = [t for t in self.tracked_stracks if t.state == 'tracked'] + activated_starcks
        self.lost_stracks = [t for t in self.lost_stracks if t.time_since_update <= self.max_time_lost] + lost_stracks
        output = [{'track_id': t.track_id, 'bbox': [int(x) for x in t.tlbr]} for t in self.tracked_stracks if t.is_activated]
        return output
    def linear_assignment(self, cost_matrix, thresh):
        if cost_matrix.size == 0: return [], [], []
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        matches = [(r, c) for r, c in zip(row_ind, col_ind) if cost_matrix[r, c] < thresh]
        u_track = [r for r in range(cost_matrix.shape[0]) if r not in [m[0] for m in matches]]
        u_detection = [c for c in range(cost_matrix.shape[1]) if c not in [m[1] for m in matches]]
        return matches, u_track, u_detection


In [ ]:
class PlayerFeatureExtractor(nn.Module):
    def __init__(self, embedding_dim=128):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, embedding_dim)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.normalize(x, p=2, dim=1)
class PlayerReID:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.feature_extractor = PlayerFeatureExtractor().to(self.device).eval()
        self.ocr = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
        self.global_players = {}
        self.next_permanent_id = 1
        self.similarity_threshold = 0.45
        self.jersey_bonus = 0.4
    def get_features(self, patch):
        hsv = cv2.cvtColor(cv2.resize(patch, (64, 64)), cv2.COLOR_BGR2HSV)
        color_hist = cv2.normalize(cv2.calcHist([hsv], [0, 1], None, [30, 32], [0, 180, 0, 256]), None).flatten()
        img_tensor = torch.from_numpy(cv2.resize(patch, (64, 64))).permute(2, 0, 1).float().div(255).unsqueeze(0).to(self.device)
        with torch.no_grad():
            deep_features = self.feature_extractor(img_tensor).cpu().numpy().flatten()
        jersey = None
        try:
            gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)
            results = self.ocr.readtext(gray, allowlist='0123456789', detail=0, paragraph=False)
            if results and results[0].isdigit() and 1 <= len(results[0]) <= 2:
                jersey = int(results[0])
        except Exception:
            pass
        return {'color': color_hist, 'deep': deep_features, 'jersey': jersey}
    def process_tracklets(self, tracklets_path, video_path, output_path):
        with open(tracklets_path, 'r') as f:
            all_tracklets = json.load(f)
        cap = cv2.VideoCapture(video_path)
        frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        long_tracks = []
        current_frame_index = -1
        for frame_data in tqdm(all_tracklets, desc='Stage 3: Re-identifying players'):
            target_index = frame_data['frame_id']
            while current_frame_index < target_index:
                ret, frame = cap.read()
                if not ret:
                    break
                current_frame_index += 1
            if current_frame_index != target_index:
                break
            frame_tracks = {'frame_id': target_index, 'players': []}
            for track in frame_data['tracks']:
                x1, y1, x2, y2 = track['bbox']
                x1 = max(0, min(frame_w - 1, int(x1)))
                y1 = max(0, min(frame_h - 1, int(y1)))
                x2 = max(0, min(frame_w, int(x2)))
                y2 = max(0, min(frame_h, int(y2)))
                if x2 <= x1 or y2 <= y1:
                    continue
                patch = frame[y1:y2, x1:x2]
                if patch.size == 0:
                    continue
                current_features = self.get_features(patch)
                best_id = None
                best_score = self.similarity_threshold
                for pid, p_info in self.global_players.items():
                    color_sim = cv2.compareHist(current_features['color'], p_info['features']['color'], cv2.HISTCMP_CORREL)
                    deep_sim = float(cosine_similarity(current_features['deep'].reshape(1, -1), p_info['features']['deep'].reshape(1, -1))[0][0])
                    sim = 0.4 * color_sim + 0.6 * deep_sim
                    if current_features['jersey'] is not None and current_features['jersey'] == p_info['features']['jersey']:
                        sim += self.jersey_bonus
                    if sim > best_score:
                        best_score = sim
                        best_id = pid
                if best_id is None:
                    best_id = self.next_permanent_id
                    self.next_permanent_id += 1
                if best_id in self.global_players:
                    alpha = 0.12
                    self.global_players[best_id]['features']['color'] = (1 - alpha) * self.global_players[best_id]['features']['color'] + alpha * current_features['color']
                    self.global_players[best_id]['features']['deep'] = (1 - alpha) * self.global_players[best_id]['features']['deep'] + alpha * current_features['deep']
                    if self.global_players[best_id]['features']['jersey'] is None and current_features['jersey'] is not None:
                        self.global_players[best_id]['features']['jersey'] = current_features['jersey']
                else:
                    self.global_players[best_id] = {'features': current_features}
                frame_tracks['players'].append({'permanent_id': best_id, 'bbox': [x1, y1, x2, y2], 'jersey': current_features['jersey']})
            long_tracks.append(frame_tracks)
        cap.release()
        with open(output_path, 'w') as f:
            json.dump(long_tracks, f, indent=2)
        return long_tracks


In [ ]:
class AdvancedEventDetector:
    def __init__(self, video_width, video_height, fps):
        self.video_width = video_width
        self.video_height = video_height
        self.fps = fps if fps and fps > 0 else 30.0
        self.player_history = {}
        self.sprint_velocity_threshold = 18.0
        self.tackle_proximity_threshold = 80
        self.fall_aspect_ratio_threshold = 1.35
        self.dribble_direction_change_threshold = 40
        self.goal_area_left = [0, 0, video_width * 0.2, video_height]
        self.goal_area_right = [video_width * 0.8, 0, video_width, video_height]
        self.cluster_size = 3
        self.cluster_radius = 160
        self.celebration_window = int(5 * self.fps)
    def _update_player_history(self, players, frame_id):
        for p in players:
            pid = p['permanent_id']
            x1, y1, x2, y2 = p['bbox']
            center = np.array([(x1 + x2) / 2, (y1 + y2) / 2])
            width = x2 - x1
            height = y2 - y1
            aspect_ratio = width / height if height > 0 else 1
            if pid not in self.player_history:
                self.player_history[pid] = []
            history = self.player_history[pid]
            velocity = np.array([0, 0])
            speed = 0
            if len(history) > 0:
                prev_center = history[-1]['center']
                velocity = center - prev_center
                speed = float(np.linalg.norm(velocity))
            history.append({'frame_id': frame_id, 'center': center, 'bbox': p['bbox'], 'velocity': velocity, 'speed': speed, 'aspect_ratio': aspect_ratio})
            if len(history) > self.fps * 3:
                self.player_history[pid] = history[-int(self.fps * 3):]
    def detect_events(self, player_tracks_path, output_path):
        with open(player_tracks_path, 'r') as f:
            all_tracks = json.load(f)
        events = []
        last_goal_entry = {}
        for frame_data in tqdm(all_tracks, desc='Stage 4: Detecting events'):
            frame_id = frame_data['frame_id']
            players = frame_data['players']
            self._update_player_history(players, frame_id)
            player_ids = [p['permanent_id'] for p in players]
            for pid in player_ids:
                hist = self.player_history.get(pid, [])
                if not hist:
                    continue
                if hist[-1]['speed'] > self.sprint_velocity_threshold:
                    events.append({'frame_id': frame_id, 'timestamp': frame_id / self.fps, 'event_type': 'sprint', 'player_id': pid})
                if len(hist) > 6:
                    v1 = hist[-6]['velocity']
                    v2 = hist[-1]['velocity']
                    a1 = v1 / np.linalg.norm(v1) if np.linalg.norm(v1) > 0 else np.array([0, 0])
                    a2 = v2 / np.linalg.norm(v2) if np.linalg.norm(v2) > 0 else np.array([0, 0])
                    ang = np.rad2deg(np.arccos(np.clip(np.dot(a1, a2), -1, 1))) if np.linalg.norm(a1) > 0 and np.linalg.norm(a2) > 0 else 0
                    if ang > self.dribble_direction_change_threshold and hist[-1]['speed'] > 5.0:
                        events.append({'frame_id': frame_id, 'timestamp': frame_id / self.fps, 'event_type': 'skill_move', 'player_id': pid})
                cx, cy = hist[-1]['center']
                if self.goal_area_right[0] < cx < self.goal_area_right[2] and self.goal_area_right[1] < cy < self.goal_area_right[3]:
                    last_goal_entry[pid] = frame_id
                if self.goal_area_left[0] < cx < self.goal_area_left[2] and self.goal_area_left[1] < cy < self.goal_area_left[3]:
                    last_goal_entry[pid] = frame_id
            centers = [self.player_history[pid][-1]['center'] for pid in self.player_history if self.player_history[pid] and self.player_history[pid][-1]['frame_id'] == frame_id]
            if len(centers) >= self.cluster_size:
                from sklearn.cluster import DBSCAN
                clustering = DBSCAN(eps=self.cluster_radius, min_samples=self.cluster_size).fit(centers)
                if len(set(clustering.labels_)) > 1:
                    for pid, entry_f in last_goal_entry.items():
                        if frame_id - entry_f < self.celebration_window:
                            events.append({'frame_id': frame_id, 'timestamp': frame_id / self.fps, 'event_type': 'goal_shot_attempt', 'player_id': pid})
                            break
            ids = list(self.player_history.keys())
            for i in range(len(ids)):
                for j in range(i + 1, len(ids)):
                    hi = self.player_history[ids[i]]
                    hj = self.player_history[ids[j]]
                    if hi and hj and hi[-1]['frame_id'] == frame_id and hj[-1]['frame_id'] == frame_id:
                        d = float(np.linalg.norm(hi[-1]['center'] - hj[-1]['center']))
                        if d < self.tackle_proximity_threshold:
                            if hi[-1]['aspect_ratio'] > self.fall_aspect_ratio_threshold:
                                events.append({'frame_id': frame_id, 'timestamp': frame_id / self.fps, 'event_type': 'tackle_fall', 'player_id': ids[i]})
                            if hj[-1]['aspect_ratio'] > self.fall_aspect_ratio_threshold:
                                events.append({'frame_id': frame_id, 'timestamp': frame_id / self.fps, 'event_type': 'tackle_fall', 'player_id': ids[j]})
        if not events and all_tracks:
            total_frames = all_tracks[-1]['frame_id'] + 1
            stride = max(1, int(self.fps * 10))
            for f in range(0, total_frames, stride):
                events.append({'frame_id': f, 'timestamp': f / self.fps, 'event_type': 'moment', 'player_id': None})
        events.sort(key=lambda e: e['frame_id'])
        unique = []
        for e in events:
            if not unique:
                unique.append(e)
            else:
                if e['frame_id'] - unique[-1]['frame_id'] >= int(self.fps * 2):
                    unique.append(e)
        with open(output_path, 'w') as f:
            json.dump(unique, f, indent=2)
        return unique


In [ ]:
class VideoAssembler:
    def __init__(self, clip_padding_seconds=1.0, max_reel_duration=300.0):
        self.clip_padding = clip_padding_seconds
        self.max_duration = max_reel_duration
        self.temp_dir = '/content/temp_clips'
    def find_scenes(self, video_path):
        video = open_video(video_path)
        scene_manager = SceneManager()
        scene_manager.add_detector(ContentDetector(threshold=27.0))
        scene_manager.detect_scenes(video, show_progress=False)
        scene_list = scene_manager.get_scene_list()
        return [(s[0].get_seconds(), s[1].get_seconds()) for s in scene_list]
    def assemble_highlight_reel(self, video_path, player_events_path, output_path, target_player_id):
        with open(player_events_path, 'r') as f:
            all_events = json.load(f)
        player_events = [e for e in all_events if e.get('player_id') == target_player_id or 'goal' in e.get('event_type', '')]
        if not player_events:
            return False, 0
        scenes = self.find_scenes(video_path)
        event_timestamps = sorted([e['timestamp'] for e in player_events])
        clips_to_extract = []
        total_duration = 0.0
        for ts in event_timestamps:
            if total_duration >= self.max_duration:
                break
            for start, end in scenes:
                if start <= ts <= end:
                    clip_duration = end - start
                    if total_duration + clip_duration <= self.max_duration:
                        if not any(abs(c['start'] - start) < 0.5 for c in clips_to_extract):
                            clips_to_extract.append({'start': max(0, start - self.clip_padding), 'end': end + self.clip_padding})
                            total_duration += clip_duration
                    break
        if not clips_to_extract:
            return False, 0
        shutil.rmtree(self.temp_dir, ignore_errors=True)
        os.makedirs(self.temp_dir, exist_ok=True)
        clip_paths = []
        for i, clip in enumerate(tqdm(clips_to_extract, desc='Extracting scene clips')):
            clip_path = os.path.join(self.temp_dir, f'clip_{i:04d}.mp4')
            command = ['ffmpeg', '-y', '-i', video_path, '-ss', str(clip['start']), '-to', str(clip['end']), '-c', 'copy', '-avoid_negative_ts', '1', clip_path]
            result = subprocess.run(command, capture_output=True, text=True)
            if result.returncode == 0:
                clip_paths.append(clip_path)
        if not clip_paths:
            return False, 0
        concat_list_path = os.path.join(self.temp_dir, 'concat_list.txt')
        with open(concat_list_path, 'w') as f:
            for path in clip_paths:
                f.write(f"file '{os.path.basename(path)}'\n")
        concat_command = ['ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', 'concat_list.txt', '-c', 'copy', output_path]
        concat_result = subprocess.run(concat_command, cwd=self.temp_dir, capture_output=True, text=True)
        if concat_result.returncode == 0:
            return True, len(player_events)
        else:
            return False, len(player_events)


In [ ]:
if 'video_path' in locals() and video_path:
    detections_path = '/content/output/detections.json'
    tracklets_path = '/content/output/tracklets.json'
    long_tracks_path = '/content/output/long_player_track.json'
    events_path = '/content/output/player_events.json'
    target_player_id = 1
    highlight_path = f'/content/output/player_{target_player_id}_highlights.mp4'
    detector = SoccerPlayerDetector()
    detections = detector.process_video(video_path, detections_path)
    tracker = ByteTrack(high_thresh=0.55, low_thresh=0.10, max_time_lost=30)
    all_tracklets = []
    for frame_data in tqdm(detections, desc='Stage 2: Tracking players'):
        tracks = tracker.update(frame_data['detections'])
        all_tracklets.append({'frame_id': frame_data['frame_id'], 'tracks': tracks})
    with open(tracklets_path, 'w') as f:
        json.dump(all_tracklets, f, indent=2)
    reid = PlayerReID()
    long_tracks = reid.process_tracklets(tracklets_path, video_path, long_tracks_path)
    cap = cv2.VideoCapture(video_path)
    video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    event_detector = AdvancedEventDetector(video_width, video_height, fps)
    player_events = event_detector.detect_events(long_tracks_path, events_path)
    assembler = VideoAssembler()
    success, num_events = assembler.assemble_highlight_reel(video_path, events_path, highlight_path, target_player_id)
    print('DETECTIONS:', len(detections))
    print('TRACK FRAMES:', len(all_tracklets))
    print('UNIQUE PLAYERS:', len(reid.global_players))
    print('EVENTS:', len(player_events))
    print('HIGHLIGHT PATH:', highlight_path if success else 'FAILED')
else:
    print('No video path available. Please run the upload cell first.')


In [ ]:
from google.colab import files
if 'success' in locals() and success and 'highlight_path' in locals() and os.path.exists(highlight_path):
    files.download(highlight_path)
if os.path.exists('/content/output/long_player_track.json'):
    files.download('/content/output/long_player_track.json')
if os.path.exists('/content/output/player_events.json'):
    files.download('/content/output/player_events.json')
print('Done.')


In [ ]:
print('PIPELINE SUMMARY')
print('=' * 50)
if 'video_path' in locals() and video_path and os.path.exists('/content/output/long_player_track.json') and os.path.exists('/content/output/player_events.json'):
    with open('/content/output/long_player_track.json', 'r') as f:
        long_tracks_summary = json.load(f)
    with open('/content/output/player_events.json', 'r') as f:
        player_events_summary = json.load(f)
    print('Video:', os.path.basename(video_path))
    print('Target player ID:', target_player_id if 'target_player_id' in locals() else 'N/A')
    print('Frames:', len(long_tracks_summary))
    print('Events:', len(player_events_summary))
    if 'success' in locals() and success and 'highlight_path' in locals() and os.path.exists(highlight_path):
        print('Highlight reel generated.')
else:
    print('Summary unavailable.')
print('=' * 50)
